# [모듈 3.1] Code Pipeline 생성 하기

## 참고 자료
- https://docs.aws.amazon.com/codepipeline/latest/userguide/pipelines-create.html


# 1. 환경 설정


In [ ]:
%store -r repository_name
%store -r code_build_project_name
%store -r account_id
%store -r region
%store -r bucket



# 2. 파라미터 설정
### 생성한 Code Pipeline Role 의 ARN 을 넣어 주세요. 아래를 지우고 넣습니다.
##### pipeline_role_name = "arn:aws:iam::XXXXXX:role/code-pipeline-gsmoon"


In [ ]:
pipeline_name = "HelloCodePipeline"
pipeline_role_name = "arn:aws:iam::XXXXXXX:role/code-pipeline-gsmoon"
branch_name = 'master'


# 3. 파이프라인 정의를  JSON 정의

In [ ]:
pipeline_dic = {
    "pipeline": {
        "name": pipeline_name,
        "roleArn": pipeline_role_name,
        "artifactStore": {
            "type": "S3",
            "location": bucket
        },
        "stages": [
            {
                "name": "Source",
                "actions": [
                    {
                        "name": "Source",
                        "actionTypeId": {
                            "category": "Source",
                            "owner": "AWS",
                            "provider": "CodeCommit",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "BranchName": branch_name,
                            "OutputArtifactFormat": "CODE_ZIP",
                            "PollForSourceChanges": "false",
                            "RepositoryName": repository_name
                        },
                        "outputArtifacts": [
                            {
                                "name": "SourceArtifact"
                            }
                        ],
                        "inputArtifacts": [],
                        "region": region,
                        "namespace": "SourceVariables"
                    }
                ]
            },
            {
                "name": "Build",
                "actions": [
                    {
                        "name": "BuildAndExecuteSageMakerPipeline",
                        "actionTypeId": {
                            "category": "Build",
                            "owner": "AWS",
                            "provider": "CodeBuild",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "ProjectName": code_build_project_name
                        },
                        "outputArtifacts": [
                            {
                                "name": "BuildArtifact"
                            }
                        ],
                        "inputArtifacts": [
                            {
                                "name": "SourceArtifact"
                            }
                        ],
                        "region": region,
                        "namespace": "BuildVariables"
                    }
                ]
            }
        ],
        "version": 1
    },
}

## 3.1. Dic 포맷을 JSON 으로 변경

In [ ]:
import json

pipeline_json = json.dumps(pipeline_dic)
json.loads(pipeline_json)

## 3.2. JSON 파일 저장

In [ ]:
json_file_path = 'src/hello_pipeline.json'
with open(json_file_path, "w") as outfile:
    outfile.write(pipeline_json)

In [ ]:
with open(json_file_path) as json_file:
    json_data = json.load(json_file)
    
json_data    

# 4. 파이프라인 실행

In [ ]:
%%sh -s {json_file_path}
json_file_path=$1
echo $json_file_path
echo $PWD
aws codepipeline create-pipeline --cli-input-json file://$json_file_path
# aws codepipeline create-pipeline --cli-input-json file://src/hello_pipeline.json

# 5. 파이프라인 실행 상태 확인

In [ ]:
%%sh -s {pipeline_name}
pipeline_name=$1
echo $pipeline_name
aws codepipeline get-pipeline-state --name $pipeline_name

In [ ]:
%store pipeline_name